In [6]:
import pandas as pd
in_out_df = pd.read_csv('aac_intakes_outcomes.csv')
in_df = pd.read_csv('aac_intakes.csv')
out_df = pd.read_csv('aac_outcomes.csv')

In [8]:
# Clean data
in_df['datetime'] = pd.to_datetime(in_df['datetime'])

### Is there an area where more pets are found?
#### Find the top 5 places where animals are found so the shelter can coordinate with local volunteers and animal control to monitor these areas.

In [5]:
location_counts = in_out_df['found_location']
top_five_locations = location_counts.value_counts()[:5]

print("Top 5 Locations where pets are found:")
top_five_locations

Top 5 Locations where pets are found:


found_location
Austin (TX)                          14311
Outside Jurisdiction                   945
Travis (TX)                            907
7201 Levander Loop in Austin (TX)      514
Del Valle (TX)                         407
Name: count, dtype: int64

### What is the average number of pets found in a month in the year 2015? Are there months where there is a higher number of animals found?
#### Knowing the number of pets the shelter might see in a month can help them gather enough resources and donations to care for the animals they receive.


In [17]:
pets_2015 = in_df[in_df['datetime'].dt.year == 2015].copy()
pets_2015['month'] = pets_2015['datetime'].dt.month
monthly_counts = pets_2015['month'].value_counts().sort_index()

average_pets_per_month = monthly_counts.mean()

average_pets_per_month

1559.3333333333333

In [18]:
monthly_counts

month
1     1198
2     1119
3     1346
4     1543
5     2094
6     2189
7     1635
8     1718
9     1591
10    1740
11    1411
12    1128
Name: count, dtype: int64

In [19]:
months_over_1559 = monthly_counts[monthly_counts > 1559]
months_over_1559

month
5     2094
6     2189
7     1635
8     1718
9     1591
10    1740
Name: count, dtype: int64

### What is the ratio of incoming pets vs. adopted pets?
#### This key metric helps the shelter know how they are doing.

In [21]:
total_incoming_pets = len(in_df)

adopted_pets = out_df[out_df['outcome_type'] == 'Adoption']
total_adopted_pets = len(adopted_pets)

adoption_ratio = total_incoming_pets / total_adopted_pets

adoption_ratio

2.3424573498480954

### What is the distribution of the types of animals in the shelter?
#### Find the count of each type of animal in the shelter.

### What are the adoption rates for specific breeds?
#### Find the top 5 dog breeds in the shelter (based on count) and then find the adoption percentage of each breed.

### What are the adoption rates for different colorings?
#### Find the top 5 colorings in the shelter (based on count) and then find the adoption percentage of each color.

### About how many animals are spayed/neutered each month?
#### This will help the shelter allocate resources and staff. Assume that all intact males and females will be spayed/neutered.